In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xgboost 

from sklearn.metrics import mean_squared_error

from xgboost import XGBRegressor, XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_absolute_percentage_error
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from sklearn.metrics import accuracy_score
from time import time
from google.colab import drive
from time import perf_counter
import xgboost as xgb
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


# Preprocess

In [ ]:
# Dataset with deletion of NaN
df_clean =  pd.read_csv('/gdrive/MyDrive/X_station_train_clean.csv', index_col=0)

# Dataset with imputation of NaN 
# df_imputation = pd.read_csv('/gdrive/MyDrive/X_station_Train_imputation.csv', index_col=0)

# 1rst Dataset with NaN
df_full_dataset = pd.read_csv('/gdrive/MyDrive/X_station_train.csv')

/usr/local/lib/python3.7/dist-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [ ]:
def m_mape(y_true,y_predict):
    n = len(y_true)
    At = np.array(y_true) + 1
    Ft = np.array(y_predict) + 1

    res = ((100/n)*(np.sum(np.abs((Ft-At)/At))))
    return res

In [ ]:
# display(df_clean[df_clean['precipitations']>0])
X_station = df_clean.copy()
Y_station_train_regr = X_station.groupby(['station_id'])['precipitations'].shift(-1)
X_station.loc[X_station.precipitations>0,'precipitations']=1
print(X_station.shape)
Y_station_train = X_station.groupby(['station_id'])['precipitations'].shift(-1)


listNan=Y_station_train[Y_station_train.isna()].index.values.tolist()
if len(listNan)>0:
  X_station = X_station.drop(listNan)
  Y_station_train = Y_station_train.drop(listNan)
  Y_station_train_regr = Y_station_train_regr.drop(listNan)


X_train = X_station[X_station['year']==2016]
listX_train=X_station[X_station['year']==2016].index.values.tolist()
y_train=Y_station_train[listX_train]
y_train_regr = Y_station_train_regr[listX_train]

X_test = X_station[X_station['year']==2017]
listX_test=X_station[X_station['year']==2017].index.values.tolist()
y_test=Y_station_train[listX_test]
y_test_regr = Y_station_train_regr[listX_test]



(2264105, 15)


In [ ]:
def normalizing_data (X, L_labels, L_labels_cos_sin, min_train, max_train):

    # Exctact & Reorder columns
    X = X[L_labels]

    # Normalize
    X = (X - min_train) / (max_train - min_train)

    # Les valeurs sont normalisées entre 0 et 1, or cos(0)=cos(2*pi) => janvier=decembre, donc la plus grande valeur (normalisée) ne doit pas être 1
    X['month_cos'] = np.cos(2*np.pi * X['month'] * 11/12)  
    X['month_sin'] = np.sin(2*np.pi * X['month'] * 11/12)
    X['hour_cos'] = np.cos(2*np.pi * X['hour'] * 23/24)
    X['hour_sin'] = np.sin(2*np.pi * X['hour'] * 23/24)
    X['wind_direction_cos'] = np.cos(2*np.pi * X['wind_direction'] * 359/360)
    X['wind_direction_sin'] = np.sin(2*np.pi * X['wind_direction'] * 359/360)

    X.drop('month', axis=1, inplace=True)
    X.drop('hour', axis=1, inplace=True)
    X.drop('wind_direction', axis=1, inplace=True)

    # Reorder columns
    X = X[L_labels_cos_sin]

    # X_station = X_station.reset_index(drop=True)

    return X

# Resampling Dataset : remove 

In [ ]:
def imalanced_remove_data(X_train,y_train):
  df_class_0 = X_train[X_train['precipitations'] == 0]

  df_class_1 = X_train[X_train['precipitations'] == 1]
  display(len(df_class_0), len(df_class_1))

  df_class_0_under = df_class_0.sample(len(df_class_1))


  X_train_class = pd.concat([df_class_0_under, df_class_1], axis=0)

  y_train_class = y_train[X_train_class.index]

  print('Random under-sampling:')

  return X_train_class,y_train_class

In [ ]:
X_train_balanced,y_train_balanced = imalanced_remove_data(X_train,y_train)
print(X_train_balanced.shape,y_train_balanced.shape)

check_for_nan = X_train_balanced.isnull().sum().sum()
print("Xtrain nan :",check_for_nan)
check_for_nan = y_train_balanced.isnull().sum().sum()
print("ytrain nan :",check_for_nan)

display(X_train_balanced["precipitations"].unique())

 #Normalize
L_labels = ['latitude', 'longitude', 'altitude', 'month', 'hour', 'wind_direction', 'wind_speed', 'temperature', 'humidity', 'dew_point', 'precipitations']
L_labels_cos_sin = ['latitude', 'longitude', 'altitude', 'month_cos', 'month_sin', 'hour_cos', 'hour_sin', 'wind_direction_cos', 'wind_direction_sin', 'wind_speed', 'temperature', 'humidity', 'dew_point', 'precipitations']

min_train = X_train[L_labels].min()
max_train = X_train[L_labels].max()

X_train_balanced = normalizing_data(X_train_balanced, L_labels, L_labels_cos_sin, min_train, max_train)
X_test_norm = X_test
X_test_norm = normalizing_data(X_test, L_labels, L_labels_cos_sin, min_train, max_train)
# display(X_train_balanced)

# Classifier

In [ ]:
my_classifier =xgb.XGBClassifier(
    max_depth=5,
    learning_rate=0.01,
    gamma=0.25,
    reg_lambda=1,
)

t1_start = perf_counter()              
my_classifier.fit(X_train_balanced, y_train_balanced)
t1_stop = perf_counter()
print("Time:", t1_stop-t1_start)

In [ ]:
X_test_norm.reset_index(drop=True, inplace=True)
t1_start = perf_counter() 
y_predict = my_classifier.predict(X_test_norm)
t1_stop = perf_counter()
print("Time:", t1_stop-t1_start)
print("My MAPE =", m_mape(y_test,y_predict))
print("MSE =",mean_squared_error(y_test,y_predict))
save = y_predict

accuracy = accuracy_score(y_test, y_predict)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# Regressor

In [ ]:
y_pluie = y_test_regr[y_predict == 1]

id = np.where(y_predict == 1)[0].tolist()
print(id)
X_pluie = X_test_norm.loc[id]

print(X_pluie.shape,y_pluie.shape)
X_train_pluie, X_test_pluie, y_train_pluie, y_test_pluie = train_test_split(X_pluie, y_pluie, test_size=0.33, random_state=42)

regr = xgb.XGBRegressor(n_estimators=20, 
                     max_depth=18,
                     gamma=3.430739184133814, 
                     min_child_weight = 8,
                     reg_alpha=180,
                     reg_lambda=0.7436396623675846,
                     random_state=123)

t1_start = perf_counter()              
regr.fit(X_train_pluie, y_train_pluie)
t1_stop = perf_counter()
print("Time:", t1_stop-t1_start)



In [ ]:
t1_start = perf_counter() 
y_predict_regressor = regr.predict(X_test_pluie)
t1_stop = perf_counter()

print(y_test_pluie.shape,y_predict_regressor.shape)
print("Time:", t1_stop-t1_start)

print("\nMAPE données juste pluie")
print("My MAPE =", m_mape(y_test_pluie,y_predict_regressor))
print("MSE =",mean_squared_error(y_test_pluie,y_predict_regressor))


for i,j in zip(id,y_predict_regressor) :
  y_predict[i]=j

print("\nMAPE toutes données")
print("My MAPE =", m_mape(y_test_regr,y_predict))
print("MSE =",mean_squared_error(y_test_regr,y_predict))

(21282,) (21282,)
Time: 0.03510145299969736

MAPE données juste pluie
My MAPE = 39.09448333068498
MSE = 1.2442115899833537

MAPE toutes données
My MAPE = 5.102892697277603
MSE = 0.18847708241439076


In [ ]:
display(y_test_regr[y_test_regr>0])
display(y_predict[y_predict>0])

# Data test


In [ ]:
X_station_test_path = '/gdrive/MyDrive/X_station_test.csv'
stations_coordinates_path = '/gdrive/MyDrive/stations_coordinates.csv'
L_labels_test  = ['station_id', 'day_id', 'latitude', 'longitude', 'altitude',         'month',        'hour', 'wind_direction', 'wind_speed', 'temperature', 'humidity', 'dew_point', 'precipitations']
X_station_test = preprocessing_X_station(X_station_test_path, stations_coordinates_path, L_labels_test)
X_test = normalizing_data (X_station_test, L_labels, L_labels_cos_sin, min_train, max_train)
display(X_test)

In [ ]:
Y_pred_test1 = my_classifier.predict(X_test)
id = np.where(Y_pred_test1 == 1)[0].tolist()
X_pluie = X_test.loc[id]
Y_pred_test2 = regr.predict(X_pluie)

for i,j in zip(id,Y_pred_test2) :
  Y_pred_test1[i]=j


Y_pred_test1 = Y_pred_test1 * (max_train['precipitations'] - min_train['precipitations']) + min_train['precipitations']
pred_merged = pd.concat([X_station_test, pd.DataFrame(Y_pred_test1, columns=['Y_pred'])], axis=1)
pred_merged = pred_merged[['station_id',	'day_id', 'Y_pred']]
pred_merged['station_day_id'] = pred_merged[['station_id', 'day_id']].astype(str).apply(lambda x: '_'.join(x), axis=1)

pred_merged.drop(['station_id',	'day_id'], axis=1, inplace=True)
pred_merged = pred_merged[['station_day_id', 'Y_pred']]

pred_merged = pred_merged.groupby('station_day_id').agg(np.sum).reset_index()

baseline_obs = pd.read_csv('/gdrive/MyDrive/Baseline_observation_test.csv')
baseline_obs.drop('Prediction', axis=1, inplace=True)

baseline_obs = baseline_obs.rename(columns={"Id": "station_day_id"})
pred_merged = pd.merge(baseline_obs, pred_merged, how='inner', on=['station_day_id'])

pred_merged = pred_merged.rename(columns={'station_day_id': "Id", 'Y_pred':"Prediction"})

display(pred_merged)

In [ ]:
pred_merged.to_csv('/gdrive/MyDrive/MLDM-Prediction/xgboostClassifer_XgboostRegressor_data_normalisee2.csv',index=False)

In [ ]:
print(len(id))

46802


In [ ]:
print(pred_merged['Prediction'].describe())
display(len(Y_pred_test1[Y_pred_test1>0]))
display(len(Y_pred_test1[Y_pred_test1==0]))

count    85140.000000
mean         0.507104
std          1.507873
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         17.925201
Name: Prediction, dtype: float64


46802

2258000